In [54]:
import cv2
import HandTrackingModule as htm
from time import sleep
from pynput.keyboard import Controller

In [13]:
cap=cv2.VideoCapture(0) 
cap.set(3,1280)
cap.set(4,720)
detector=htm.handDetector(detectionCon=0.8)
keys = [["Q", "W", "E", "R", "T", "Y", "U", "I", "O", "P"],
        ["A", "S", "D", "F", "G", "H", "J", "K", "L", ";"],
        ["Z", "X", "C", "V", "B", "N", "M", ",", ".", "/"]]

In [19]:
def drawAll(img,buttonList):
    for button in buttonList:
        x,y=button.pos
        w,h=button.size
        cv2.rectangle(img,button.pos,(x+w,y+h),(255,0,255),cv2.FILLED)
        cv2.putText(img,button.text,(x+20,y+65),cv2.FONT_HERSHEY_PLAIN,4,(175,0,175),4)
    return img
    


In [20]:
class Button():
    def __init__(self,pos,text,size=[85, 85]):
        self.pos=pos
        self.text=text
        self.size=size
        

In [ ]:
buttonList=[]
finaltext=''
keyborad= Controller()  
for i in range(len(keys)):
    for x,key in enumerate(keys[i]):
        buttonList.append(Button([100*x+50,100*i+50],key))
while True:
    # 1.read image 
    succes,img=cap.read()
    #flip the image  1=horzintely and 0 vertical 
    img=cv2.flip(img,1)
    img=detector.findHands(img)
    lmList,bbox=detector.findPosition(img)
    img=drawAll(img,buttonList)
    if lmList:
        for button in buttonList:
            x, y = button.pos
            w, h = button.size
            if x < lmList[8][1] < x + w and y < lmList[8][2] < y + h:
                cv2.rectangle(img, (x - 5, y - 5), (x + w + 5, y + h + 5), (175, 0, 175), cv2.FILLED)
                cv2.putText(img, button.text, (x + 20, y + 65),
                            cv2.FONT_HERSHEY_PLAIN, 4, (255, 255, 255), 4)
                l,_,_=detector.findDistance(8,12,img)
                print(l)
                if l<50:
                    keyborad.press(button.text)
                    cv2.rectangle(img, button.pos, (x + w, y + h), (0, 255, 0), cv2.FILLED)
                    cv2.putText(img, button.text, (x + 20, y + 65),
                                cv2.FONT_HERSHEY_PLAIN, 4, (255, 255, 255), 4)
                    finaltext+=button.text
                    sleep(0.15)
        fingers=detector.fingersUp()
        print(fingers)
        if all (x <= 0 for x in fingers):
            finaltext=''       
    cv2.rectangle(img, (50, 350), (700, 450), (175, 0, 175), cv2.FILLED)
    cv2.putText(img,finaltext, (60, 430),
                cv2.FONT_HERSHEY_PLAIN, 5, (255, 255, 255), 5)
    #showing image
    cv2.imshow("image",img)
    cv2.waitKey(1)